**Capstone Project:**

**Dynamic pricing model-1** :
Baseline Linear Model

we are create data-driven
pricing engine for 14 parking spaces using real-time data streams.

we use pathway and bokeh For simulate real-time data streaming and creating interactive, live-updating charts.

In [21]:
!pip install pathway bokeh

Now, import numpy and panda for our project.

In [22]:
import numpy as np

In [23]:
import pandas as pd


Now import pathway

In [24]:
import pathway as pw

class ParkingSchema(pw.Schema):
    timestamp: str
    lot_id: str
    capacity: int
    occupancy: int

table = pw.io.csv.read(
    "dataset.csv",
    schema=ParkingSchema,
    autocommit_duration_ms=1000
)


**Now we are here defining pricing logic **

In [25]:
ALPHA = 2
BASE_PRICE = 10

@pw.udf
def compute_price(occupancy, capacity):
    ratio = occupancy / capacity
    return round(BASE_PRICE + ALPHA * ratio, 2)


BASE_PRICE = $10 for every parking lot.

occupancy / capacity = how full the lot is (0 to 1).

ALPHA controls how much price rises with demand.


In [26]:
table_with_price = table.select(
    lot_id=table.lot_id,
    timestamp=table.timestamp,
    occupancy=table.occupancy,
    capacity=table.capacity,
    price=compute_price(table.occupancy, table.capacity)
)


In [27]:
import time
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook

We use **Bokeh’s** ColumnDataSource to stream data into the plot.

x is time, y is price.

The plot will be updated as new prices arrive for Lot_1.

In [30]:
df = pd.read_csv("dataset.csv")
print(df.columns)

Index(['ID', 'SystemCodeNumber', 'Capacity', 'Latitude', 'Longitude',
       'Occupancy', 'VehicleType', 'TrafficConditionNearby', 'QueueLength',
       'IsSpecialDay', 'LastUpdatedDate', 'LastUpdatedTime'],
      dtype='object')


In [ ]:
def run_stream_all_lots():
    for timestamp in sorted(df["timestamp"].unique()):
        current_time_data = df[df["timestamp"] == timestamp]

        for _, row in current_time_data.iterrows():
            lot = row["lot_id"]
            ts = pd.to_datetime(row["timestamp"])
            price = row["price"]

            data_sources[lot].stream(
                {"x": [ts], "y": [price]}, rollover=50
            )

        push_notebook(handle=handle)
        time.sleep(1)  # simulate 1 second delay per time step

run_stream_all_lots()